# Neural Network

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pickle
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import binary_crossentropy
from keras.optimizers import rmsprop
from keras.metrics import categorical_crossentropy
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


# environment settings
data_path = 'Data/'

# Deserialize previously saved data from "preprocessing"
with open(data_path+'train_pp.obj', 'rb') as train_pp, \
open(data_path+'test_pp.obj','rb') as test_pp:
    train_df = pickle.load(train_pp)
    test_df = pickle.load(test_pp)

In [4]:
#Separate training set from target variables
X_train = train_df.drop("Survived", axis=1)
y_train = train_df["Survived"]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Title
0,1,3,0,22.000000,1,0,7.2500,0,1,6
1,2,1,1,38.000000,1,0,71.2833,0,0,1
2,3,3,1,26.000000,0,0,7.9250,0,1,2
3,4,1,1,35.000000,1,0,53.1000,0,1,1
4,5,3,0,35.000000,0,0,8.0500,0,1,6
5,6,3,0,21.222694,0,0,8.4583,1,0,6
6,7,1,0,54.000000,0,0,51.8625,0,1,6
7,8,3,0,2.000000,3,1,21.0750,0,1,3
8,9,3,1,27.000000,0,2,11.1333,0,1,1
9,10,2,1,14.000000,1,0,30.0708,0,0,1


In [5]:
#Function that creates neural net
def create_model():
    model = Sequential([
        Dense(16,input_shape = (X_train.shape[1],), activation = 'relu',kernel_regularizer=regularizers.l2(0.01)),
        Dense(32,activation = 'relu',kernel_regularizer=regularizers.l2(0.01)),
        Dense(32,activation = 'relu',kernel_regularizer=regularizers.l2(0.01),),
        Dense(1,activation = 'sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=rmsprop(lr=0.001), metrics = ['accuracy'])
    return model

In [6]:
#create neural net (for evaluation of performance)
modelSk = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=2)

#evaluate with 10-folds
kfold = KFold(n_splits=5, shuffle = True, random_state=7)
results = cross_val_score(modelSk,X_train,y=y_train, cv=kfold,verbose=2)

[CV]  ................................................................
Epoch 1/150
 - 0s - loss: 2.0673 - acc: 0.5421
Epoch 2/150
 - 0s - loss: 1.5988 - acc: 0.5955
Epoch 3/150
 - 0s - loss: 1.3704 - acc: 0.6194
Epoch 4/150
 - 0s - loss: 1.3769 - acc: 0.5913
Epoch 5/150
 - 0s - loss: 1.3144 - acc: 0.5927
Epoch 6/150
 - 0s - loss: 1.1103 - acc: 0.6348
Epoch 7/150
 - 0s - loss: 1.1283 - acc: 0.6110
Epoch 8/150
 - 0s - loss: 1.0924 - acc: 0.6334
Epoch 9/150
 - 0s - loss: 1.0329 - acc: 0.6447
Epoch 10/150
 - 0s - loss: 1.0049 - acc: 0.6503
Epoch 11/150
 - 0s - loss: 0.9784 - acc: 0.6250
Epoch 12/150
 - 0s - loss: 0.9212 - acc: 0.6756
Epoch 13/150
 - 0s - loss: 0.8837 - acc: 0.6643
Epoch 14/150
 - 0s - loss: 0.8844 - acc: 0.6699
Epoch 15/150
 - 0s - loss: 0.8570 - acc: 0.6713
Epoch 16/150
 - 0s - loss: 0.8245 - acc: 0.6868
Epoch 17/150
 - 0s - loss: 0.8154 - acc: 0.6713
Epoch 18/150
 - 0s - loss: 0.7845 - acc: 0.6868
Epoch 19/150
 - 0s - loss: 0.7676 - acc: 0.6896
Epoch 20/150
 - 0s - loss:

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.5s remaining:    0.0s


Epoch 1/150
 - 0s - loss: 4.2224 - acc: 0.5526
Epoch 2/150
 - 0s - loss: 2.3036 - acc: 0.5792
Epoch 3/150
 - 0s - loss: 2.2482 - acc: 0.5877
Epoch 4/150
 - 0s - loss: 1.8907 - acc: 0.6199
Epoch 5/150
 - 0s - loss: 1.6474 - acc: 0.6438
Epoch 6/150
 - 0s - loss: 1.8017 - acc: 0.6171
Epoch 7/150
 - 0s - loss: 1.6871 - acc: 0.6199
Epoch 8/150
 - 0s - loss: 1.4526 - acc: 0.6410
Epoch 9/150
 - 0s - loss: 1.5533 - acc: 0.6269
Epoch 10/150
 - 0s - loss: 1.4532 - acc: 0.6101
Epoch 11/150
 - 0s - loss: 1.2905 - acc: 0.6620
Epoch 12/150
 - 0s - loss: 1.1649 - acc: 0.6844
Epoch 13/150
 - 0s - loss: 1.1157 - acc: 0.6508
Epoch 14/150
 - 0s - loss: 1.0828 - acc: 0.6704
Epoch 15/150
 - 0s - loss: 1.0287 - acc: 0.6760
Epoch 16/150
 - 0s - loss: 1.0364 - acc: 0.6648
Epoch 17/150
 - 0s - loss: 0.9609 - acc: 0.6872
Epoch 18/150
 - 0s - loss: 0.9582 - acc: 0.6802
Epoch 19/150
 - 0s - loss: 0.9570 - acc: 0.6816
Epoch 20/150
 - 0s - loss: 0.9427 - acc: 0.6732
Epoch 21/150
 - 0s - loss: 0.8963 - acc: 0.6634
E

 - 0s - loss: 0.7083 - acc: 0.6620
Epoch 19/150
 - 0s - loss: 0.7029 - acc: 0.6550
Epoch 20/150
 - 0s - loss: 0.6914 - acc: 0.6816
Epoch 21/150
 - 0s - loss: 0.6776 - acc: 0.6971
Epoch 22/150
 - 0s - loss: 0.6824 - acc: 0.6690
Epoch 23/150
 - 0s - loss: 0.6612 - acc: 0.6816
Epoch 24/150
 - 0s - loss: 0.6662 - acc: 0.6942
Epoch 25/150
 - 0s - loss: 0.6691 - acc: 0.6774
Epoch 26/150
 - 0s - loss: 0.6456 - acc: 0.7013
Epoch 27/150
 - 0s - loss: 0.6551 - acc: 0.7055
Epoch 28/150
 - 0s - loss: 0.6529 - acc: 0.6942
Epoch 29/150
 - 0s - loss: 0.6395 - acc: 0.7041
Epoch 30/150
 - 0s - loss: 0.6335 - acc: 0.7181
Epoch 31/150
 - 0s - loss: 0.6286 - acc: 0.7279
Epoch 32/150
 - 0s - loss: 0.6361 - acc: 0.7125
Epoch 33/150
 - 0s - loss: 0.6258 - acc: 0.7391
Epoch 34/150
 - 0s - loss: 0.6307 - acc: 0.7111
Epoch 35/150
 - 0s - loss: 0.6200 - acc: 0.7209
Epoch 36/150
 - 0s - loss: 0.6191 - acc: 0.7153
Epoch 37/150
 - 0s - loss: 0.6159 - acc: 0.7321
Epoch 38/150
 - 0s - loss: 0.6275 - acc: 0.7433
Epoch

 - 0s - loss: 0.6563 - acc: 0.7433
Epoch 36/150
 - 0s - loss: 0.6485 - acc: 0.7237
Epoch 37/150
 - 0s - loss: 0.6441 - acc: 0.7363
Epoch 38/150
 - 0s - loss: 0.6471 - acc: 0.7461
Epoch 39/150
 - 0s - loss: 0.6383 - acc: 0.7405
Epoch 40/150
 - 0s - loss: 0.6545 - acc: 0.7279
Epoch 41/150
 - 0s - loss: 0.6466 - acc: 0.7293
Epoch 42/150
 - 0s - loss: 0.6409 - acc: 0.7447
Epoch 43/150
 - 0s - loss: 0.6302 - acc: 0.7602
Epoch 44/150
 - 0s - loss: 0.6374 - acc: 0.7307
Epoch 45/150
 - 0s - loss: 0.6370 - acc: 0.7475
Epoch 46/150
 - 0s - loss: 0.6405 - acc: 0.7391
Epoch 47/150
 - 0s - loss: 0.6282 - acc: 0.7405
Epoch 48/150
 - 0s - loss: 0.6275 - acc: 0.7489
Epoch 49/150
 - 0s - loss: 0.6163 - acc: 0.7546
Epoch 50/150
 - 0s - loss: 0.6122 - acc: 0.7574
Epoch 51/150
 - 0s - loss: 0.6133 - acc: 0.7588
Epoch 52/150
 - 0s - loss: 0.6067 - acc: 0.7546
Epoch 53/150
 - 0s - loss: 0.6049 - acc: 0.7405
Epoch 54/150
 - 0s - loss: 0.6130 - acc: 0.7433
Epoch 55/150
 - 0s - loss: 0.5973 - acc: 0.7686
Epoch

 - 0s - loss: 0.5794 - acc: 0.7728
Epoch 53/150
 - 0s - loss: 0.5822 - acc: 0.7630
Epoch 54/150
 - 0s - loss: 0.5700 - acc: 0.7728
Epoch 55/150
 - 0s - loss: 0.5731 - acc: 0.7728
Epoch 56/150
 - 0s - loss: 0.5683 - acc: 0.7658
Epoch 57/150
 - 0s - loss: 0.5744 - acc: 0.7700
Epoch 58/150
 - 0s - loss: 0.5713 - acc: 0.7588
Epoch 59/150
 - 0s - loss: 0.5660 - acc: 0.7658
Epoch 60/150
 - 0s - loss: 0.5798 - acc: 0.7616
Epoch 61/150
 - 0s - loss: 0.5607 - acc: 0.7784
Epoch 62/150
 - 0s - loss: 0.5827 - acc: 0.7630
Epoch 63/150
 - 0s - loss: 0.5544 - acc: 0.7672
Epoch 64/150
 - 0s - loss: 0.5668 - acc: 0.7630
Epoch 65/150
 - 0s - loss: 0.5445 - acc: 0.7784
Epoch 66/150
 - 0s - loss: 0.5684 - acc: 0.7728
Epoch 67/150
 - 0s - loss: 0.5598 - acc: 0.7644
Epoch 68/150
 - 0s - loss: 0.5671 - acc: 0.7742
Epoch 69/150
 - 0s - loss: 0.5509 - acc: 0.7784
Epoch 70/150
 - 0s - loss: 0.5542 - acc: 0.7728
Epoch 71/150
 - 0s - loss: 0.5556 - acc: 0.7812
Epoch 72/150
 - 0s - loss: 0.5419 - acc: 0.7742
Epoch

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.2min finished


In [7]:
print("5-fold cross validation average accuracy: %.3f" % (results.mean()))

5-fold cross validation average accuracy: 0.765


In [8]:
#create neural net (actual neural net on whole training data)
model = create_model()
model.fit(x=X_train, y=y_train, batch_size=10, epochs = 150, verbose = 2)

Epoch 1/150
 - 0s - loss: 1.9219 - acc: 0.5286
Epoch 2/150
 - 0s - loss: 1.3466 - acc: 0.5825
Epoch 3/150
 - 0s - loss: 1.2659 - acc: 0.6285
Epoch 4/150
 - 0s - loss: 1.1228 - acc: 0.6150
Epoch 5/150
 - 0s - loss: 1.1592 - acc: 0.6061
Epoch 6/150
 - 0s - loss: 1.0246 - acc: 0.6162
Epoch 7/150
 - 0s - loss: 0.9398 - acc: 0.6453
Epoch 8/150
 - 0s - loss: 0.9566 - acc: 0.6184
Epoch 9/150
 - 0s - loss: 0.8567 - acc: 0.6487
Epoch 10/150
 - 0s - loss: 0.8129 - acc: 0.6824
Epoch 11/150
 - 0s - loss: 0.8188 - acc: 0.6498
Epoch 12/150
 - 0s - loss: 0.8004 - acc: 0.6700
Epoch 13/150
 - 0s - loss: 0.7744 - acc: 0.6835
Epoch 14/150
 - 0s - loss: 0.7752 - acc: 0.6599
Epoch 15/150
 - 0s - loss: 0.7518 - acc: 0.6779
Epoch 16/150
 - 0s - loss: 0.7383 - acc: 0.6891
Epoch 17/150
 - 0s - loss: 0.7071 - acc: 0.7003
Epoch 18/150
 - 0s - loss: 0.7233 - acc: 0.6970
Epoch 19/150
 - 0s - loss: 0.7073 - acc: 0.6958
Epoch 20/150
 - 0s - loss: 0.7003 - acc: 0.7116
Epoch 21/150
 - 0s - loss: 0.6834 - acc: 0.7059
E

In [30]:
y_out = pd.DataFrame(data = model.predict(test_df), columns=["Survived"])
y_out = y_out["Survived"].apply(lambda x: x-x if x>0.5 else x+(1-x)).astype(int)
df_out = pd.concat([test_df["PassengerId"],y_out], axis=1 )
df_out.to_csv(data_path+"outputs/nn-res.csv",sep=",",index=False)
